In [1]:
import re
import os

from tqdm import tqdm, tnrange
import json
import pandas as pd
import os
from collections import namedtuple

import utils
import parsers
from nltk.corpus import wordnet as wn
import copy
import numpy as np

import tensorflow.keras as K


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
training_file_path = '../resources/WSD_Evaluation_Framework/Training_Corpora/SemCor/semcor.data.xml'
gold_file_path =  '../resources/WSD_Evaluation_Framework/Training_Corpora/SemCor/semcor.gold.key.txt'

training_file_path = '../resources/WSD_Evaluation_Framework/Evaluation_Datasets/semeval2015/semeval2015.data.xml'
gold_file_path =  '../resources/WSD_Evaluation_Framework/Evaluation_Datasets/semeval2015/semeval2015.gold.key.txt'

training_file_path_dev = '../resources/WSD_Evaluation_Framework/Evaluation_Datasets/semeval2013/semeval2013.data.xml'
gold_file_path_dev = '../resources/WSD_Evaluation_Framework/Evaluation_Datasets/semeval2013/semeval2013.gold.key.txt'

fine_senses_vocab_path = '../resources/semcor.vocab.WordNet.json'
wordnet_domains_vocab_path = '../resources/semcor.vocab.WordNetDomain.json'
lexicographer_vocab_path = '../resources/semcor.vocab.LexNames.json'

input_vocab_path = '../resources/semcor.input.vocab.json'
input_antivocab_path = '../resources/semcor.leftout.vocab.json'
embedding_size = 64
batch_size = 64
LEARNING_RATE = 0.1
N_EPOCHS = 10
PADDING_SIZE = 30
print_model = False


#loading dict
senses = utils.json_vocab_reader(fine_senses_vocab_path)
wordnet_domains_vocabulary = utils.json_vocab_reader(wordnet_domains_vocab_path)
lexicographer_vocabulary = utils.json_vocab_reader(lexicographer_vocab_path)

inputs, antivocab = utils.json_vocab_reader(input_vocab_path, input_antivocab_path)
output_vocab = utils.merge_vocabulary(senses, inputs)
output_vocab2 = utils.merge_vocabulary(wordnet_domains_vocabulary, inputs)
output_vocab3 = utils.merge_vocabulary(lexicographer_vocabulary, inputs)



reverse_output_vocab =  dict((v, k) for k, v in output_vocab.items())

K.backend.clear_session()


In [3]:
import generatorMultitask

In [7]:
train_generator = generatorMultitask.get(batch_size = 64,
                                resources_path = '../resources',
                                training_file_path = training_file_path,
                                gold_file_path = gold_file_path,
                                antivocab = antivocab,
                                output_vocab = output_vocab,
                                output_vocab2 = output_vocab2,
                                output_vocab3 = output_vocab3,
                                PADDING_SIZE = PADDING_SIZE)

validation_generator = generatorMultitask.get(batch_size = 64,
                                resources_path = '../resources',
                                training_file_path = training_file_path_dev,
                                gold_file_path = gold_file_path_dev,
                                antivocab = antivocab,
                                output_vocab = output_vocab,
                                output_vocab2 = output_vocab2,
                                output_vocab3 = output_vocab3,
                                PADDING_SIZE = PADDING_SIZE)


In [8]:
def Multitask(vocab_size, embedding_size, hidden_size, PADDING_SIZE, LEARNING_RATE, INPUT_DROPOUT, LSTM_DROPOUT,RECURRENT_DROPOUT, N_EPOCHS):
    print("Creating MultitaskKERAS model")
    inputs = K.layers.Input(shape=(PADDING_SIZE,))
    embeddings = K.layers.Embedding(vocab_size['senses'],
                                    embedding_size,
                                    mask_zero=True,
                                    name = 'embedding')(inputs)


    BI_LSTM = (K.layers.Bidirectional(
               K.layers.LSTM(hidden_size, dropout = LSTM_DROPOUT,
                             recurrent_dropout = RECURRENT_DROPOUT,
                             return_sequences=True,
                             kernel_regularizer=K.regularizers.l2(0.01),
                             activity_regularizer=K.regularizers.l1(0.01)
                            ), name = 'Bi-directional_LSTM'))(embeddings)

    predictions_1 = K.layers.TimeDistributed(K.layers.Dense(
        vocab_size['senses'], activation='softmax', name='senses'))(BI_LSTM)
    
    predictions_2 = K.layers.TimeDistributed(K.layers.Dense(
        vocab_size['wordnet_domains'], activation='softmax', name = 'wordnet_domains'))(BI_LSTM)
    
    predictions_3 = K.layers.TimeDistributed(K.layers.Dense(
        vocab_size['lexicographer'],activation='softmax', name = 'lexicographer'))(BI_LSTM)


    model = K.models.Model(inputs=[inputs], outputs=[predictions_1,
                                                     predictions_2,
                                                     predictions_3])

    model.compile(loss = "sparse_categorical_crossentropy",#,"sparse_categorical_crossentropy","sparse_categorical_crossentropy"],
                  optimizer = K.optimizers.Adam(lr=LEARNING_RATE, decay = 0.001/N_EPOCHS, amsgrad=False),
                  metrics = ['acc'])

    return model


In [18]:
K.backend.clear_session()

In [10]:
vocab_sizes = {"senses": len(output_vocab),
               "wordnet_domains": len(output_vocab2),
               "lexicographer": len(output_vocab3)}

In [11]:
MultiTaskModelNetwork = Multitask(vocab_size = vocab_sizes,
                                embedding_size = embedding_size,
                                hidden_size = 32,
                                PADDING_SIZE = PADDING_SIZE,
                                LEARNING_RATE = LEARNING_RATE,
                                INPUT_DROPOUT = 0.25,
                                LSTM_DROPOUT = 0.45,
                                RECURRENT_DROPOUT = 0.30,
                                N_EPOCHS = N_EPOCHS)

Creating MultitaskKERAS model


In [17]:

MultiTaskModelNetwork = models.Multitask(vocab_size = vocab_sizes,
                                embedding_size = embedding_size,
                                hidden_size = 32,
                                PADDING_SIZE = PADDING_SIZE,
                                LEARNING_RATE = LEARNING_RATE,
                                INPUT_DROPOUT = 0.25,
                                LSTM_DROPOUT = 0.45,
                                RECURRENT_DROPOUT = 0.30,
                                N_EPOCHS = N_EPOCHS)

In [19]:
BasicModelNetwork = models.Basic(vocab_size = len(output_vocab),
                                embedding_size = embedding_size,
                                hidden_size = 32,
                                PADDING_SIZE = PADDING_SIZE,
                                LEARNING_RATE = LEARNING_RATE,
                                INPUT_DROPOUT = 0.25,
                                LSTM_DROPOUT = 0.45,
                                RECURRENT_DROPOUT = 0.30,
                                N_EPOCHS = N_EPOCHS)


Creating KERAS model


In [20]:
K.utils.plot_model(BasicModelNetwork, to_file='../report/img/basic.pdf')

In [14]:
import time
if not os.path.exists('../resources/logging'):
    os.mkdir('../resources/logging')
model_name = time.strftime('%Y-%m-%d_%H:%M:%S_%z')
cbk = K.callbacks.TensorBoard('../resources/logging/keras_model_'+model_name)

early_stopping = K.callbacks.EarlyStopping(monitor='val_loss',
                              min_delta=0,
                              patience=1,
                              verbose=1, mode='auto')
csv_logger = K.callbacks.CSVLogger('../resources/logging/keras_model_'+model_name+'.log')
model_checkpoint = K.callbacks.ModelCheckpoint(filepath = '../resources/logging/keras_model_'+model_name+'.h5',
                                               monitor='val_precision',
                                               verbose=2,
                                               save_best_only=True,
                                               save_weights_only=True,
                                               mode='auto', period=1)

In [15]:
train_len = generatorMultitask.__len__(training_file_path, batch_size)
val_len = generatorMultitask.__len__(training_file_path_dev, batch_size)

In [16]:
MultiTaskModelNetwork.fit_generator(train_generator, 
                                steps_per_epoch=train_len,
                                epochs=5, 
                                verbose=1,
                                callbacks=[cbk, early_stopping],
                                validation_data=validation_generator,
                                validation_steps=val_len,
                                class_weight=None,
                                max_queue_size=10,
                                workers=-1, 
                                use_multiprocessing=True,
                                shuffle=False,
                                initial_epoch=0)


models.save_model(model = BasicModelNetwork, model_name = model_name)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/5
2/2 [==============================] - 68s 34s/step - loss: 31.3608 - time_distributed_loss: 10.6404 - time_distributed_1_loss: 9.8269 - time_distributed_2_loss: 9.8181 - time_distributed_acc: 0.0076 - time_distributed_1_acc: 0.0868 - time_distributed_2_acc: 0.0852 - val_loss: 22.5992 - val_time_distributed_loss: 8.2868 - val_time_distributed_1_loss: 6.9529 - val_time_distributed_2_loss: 6.8783 - val_time_distributed_acc: 0.2669 - val_time_distributed_1_acc: 0.2499 - val_time_distributed_2_acc: 0.2499
Epoch 2/5
2/2 [==============================] - 51s 25s/step - loss: 14.2050 - time_distributed_loss: 5.2651 - time_distributed_1_loss: 3.9290 - time_distributed_2_loss: 4.1355 - time_distributed_acc: 0.4046 - time_distributed_1_acc: 0.3974 - time_distributed_2_acc: 0.3974 - val_loss: 18.9733 - val_time_distributed_loss: 6.4162 - val_time_distributed_1_loss: 5.3543 - val_time_distributed_2_loss: 5.5258 - val_time_distributed_acc: 0.2823 - val_time_distributed_1_acc: 0.2850 - va

NameError: name 'models' is not defined

In [ ]:
val_len

In [23]:
580*30/60/60


4.833333333333333